In [12]:
import plotly.express as px
from scrap import *

In [13]:
dropdown_categories = [val for val in list(get_fund_list().keys())]

dropdown_value = dropdown_categories[:4]

In [ ]:
data = pd.DataFrame(columns=['Date'])

for val in dropdown_value:
    print(val)

    df = fund_download(val)
    df.columns = ['Date', val]
    data = pd.merge(data, df, how='outer', on='Date')

S&P 500 UCITS ETF
Plotting dataframe...
U.S. Equity Index Fund
Plotting dataframe...
FTSE Developed World ex-U.K. Equity Index Fund
Plotting dataframe...
FTSE All-World UCITS ETF
Downloading...
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No cookies popup detected
-------- RETRY --------
No coo

In [ ]:
data

In [ ]:
df_perc = data.groupby(data.Date.dt.year).mean().reset_index()
df_perc

In [ ]:
d = df_perc.melt(id_vars=['Date']).dropna()
d

In [ ]:
# df = px.data.tips()
fig = px.bar(d, x="Date", y="value", facet_col="variable")
fig.show()

In [20]:
df_perc

funds,Date,S&P 500 UCITS ETF,FTSE Developed World ex-U.K. Equity Index Fund,FTSE Developed World UCITS ETF
0,2009,NaN,116.492224,NaN
1,2010,NaN,133.215630,NaN
2,2011,NaN,141.366448,NaN
3,2012,NaN,147.801441,NaN
4,2013,NaN,180.501749,NaN
5,2014,NaN,197.961091,NaN
6,2015,NaN,221.972795,NaN
7,2016,NaN,251.603944,NaN
8,2017,NaN,313.771344,NaN
9,2018,NaN,337.714296,NaN


In [18]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
